In [1]:
url = input()

In [2]:
pdfs = dict()

In [3]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from pypdf import PdfReader
import os

def get_text_from_pdf(pdf_url):
	response = requests.get(pdf_url, timeout=5, stream=True)

	with open('temp.pdf', 'wb') as f:
		f.write(response.content)

	reader = PdfReader('temp.pdf')
	text = ''.join([page.extract_text() for page in reader.pages])

	if (text == ''):
		print('Fallback to ocr, path = ' + pdf_url)
		import ocrmypdf

		ocrmypdf.ocr('temp.pdf', 'temp_ocr.pdf', language='rus', deskew=True)
		reader = PdfReader('temp_ocr.pdf')
		text = ''.join([page.extract_text() for page in reader.pages])
	
	return text

def save_pdf_to_txt(pdf_url):
	text = get_text_from_pdf(pdf_url)
	
	path = '/'.join(pdf_url.replace('https://', '').split('/')[:-1]) + '/' + pdf_url.split('/')[-1].replace('.pdf', '.txt')

	os.makedirs(os.path.dirname(path), exist_ok=True)
	with open(path, 'w') as f:
		f.write(text)

In [4]:
def get_links(url, level = 0):
	
	print(url, level)

	try:
		page = requests.get(url, timeout=5)
		soup = BeautifulSoup(page.content, 'html.parser')
		
		for pdf_link in soup.find_all('a', href=True):
			if pdf_link['href'].endswith('.pdf'):
				pdf_url = urljoin(url, pdf_link['href'])
				save_pdf_to_txt(pdf_url)

		for link in soup.find_all('a', href=True):

			href = link.get('href')
			if href is None:
				continue
	
			elif level < 2 and '.' not in href:
				get_links(urljoin(url, href), level + 1)
	except Exception as e:
		print(e)
		return

In [ ]:
get_links(url)

https://pstu.ru/sveden/education/ 0
Fallback to ocr, path = https://pstu.ru/files/file/Abitur/БУПы/2020/{04dfb049-eb61-4f47-975e-fe7a239b32ae}.pdf


Output()